# Parallel Computing

In [2]:
'''
Sempahore example
'''
class Semaphore():
    def __init__(self, max_available):
        self.cv = threading.Condition()
        self.MAX_AVAILABLE = max_available
        self.taken = 0
    
    def acquire(self):
        self.cv.acquire()
        while (self.taken == self.MAX_AVAILABLE):
            self.cv.wait()
        self.taken += 1
        self.cv.release()
        
    def release(self):
        self.cv.acquire()
        self.taken -= 1
        self.cv.notify()
        self.cv.release()

**Question 19.3**: Implement synchronization for two interleaving threads

In [2]:
"""
Book answer analysis
    + reviewing comments
"""
import threading
class OddEvenMonitor(threading.Condition):
    
    ODD_TURN = True
    EVEN_TURN = False
    
    def __init__(self):
        super().__init__()
        self.turn = self.ODD_TURN  # since 1 is odd
    
    def wait_turn(self, old_turn):
        with self:
            while self.turn != old_turn:
                self.wait()
                
    def toggle_turn(self):
        with self:
            self.turn ^= True
            self.notify()
    

class OddThread(threading.Thread):
    def __init__(self, monitor):
        super().__init__()
        self.monitor = monitor
        
    def run(self):
        for i in range(1, 101, 2):
            self.monitor.wait_turn(OddEvenMonitor.ODD_TURN) # declares it's waiting for odd_turn
            print(i)
            self.monitor.toggle_turn() # switchs monitor to even_turn after it's done printing

            
class EvenThread(threading.Thread):
    def __init__(self, monitor):
        super().__init__()
        self.monitor = monitor
    
    def run(self):
        for i in range(2, 101, 2):
            self.monitor.wait_turn(OddEvenMonitor.EVEN_TURN)
            print(i)
            self.monitor.toggle_turn()